In [1]:
import glob
import os
import pandas as pd

from langdetect import detect
import nltk
#nltk.download()   # comment after first download
from nltk.tokenize import wordpunct_tokenize, MWETokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim import corpora
import string
from numbers import Number
from pprint import pprint
import logging
import operator

pd.options.display.max_rows = 30

In [2]:
keywords_chosen = '13_FSDS_Goals_Keywords_EN.csv'

data_folder = './Accounts/output_chelsea/*.csv'

CSV_COLUMNS = ['caption_cleaned', 'hashtags']

In [3]:
# create output directory
outputDir = os.path.dirname(data_folder).replace('output_chelsea', 'q3q4_output_chelsea') + '/'
if not os.path.exists(outputDir):
    os.makedirs(outputDir)

In [4]:
# set of punctuations to remove from text
exclude = set(string.punctuation)

In [5]:
stopWords = set(stopwords.words('english'))
stopWords.add('theyre')   # an informal spelling

lemma = WordNetLemmatizer()    # NLTK English lemmatizer

# detect_lang function can be use to check the percentage of non English posts
# note that missing value NaN can be detected as many different languages such english, spanish or italian
def detect_lang(text):
    try:
        lang = detect(text)
    except:
        return 'error'
    return lang

# lemmatize_keywords also clears 'nan' from input keyword list file
# lemmatization is conducted based on context, some words may not get lemmatized, 
# e.g. "local eating" does not get lemmatized to "local eat"
def lemmatize_keywords(col):
    if str(col).lower() == 'nan':
        return ''
    return '_'.join(lemma.lemmatize(word).lower() for word in col.replace('’', '\'').replace('.', '').split()) #


In [6]:
# load keywords list
pd.options.display.max_rows = 100
keywords_df = pd.read_csv(keywords_chosen, encoding='latin-1')   # "ISO-8859-1"
KEYWORDS_COLS = keywords_df.columns
lemma_keywords_df = pd.DataFrame(columns=KEYWORDS_COLS)
category_dict = {}
keywords_list = set()
for col in KEYWORDS_COLS:
    lemma_keywords_df[col] = keywords_df[col].astype(str).apply(lemmatize_keywords)
    category_dict[col] = set(lemma_keywords_df[col].tolist())
    category_dict[col].remove('')
    keywords_list = keywords_list.union(category_dict[col])
display(lemma_keywords_df)

# if there are punctuations in the keywords list, these punctuation will be kept regardless of puncturation removal step
for word in keywords_list:
    for char in word:
        if char in exclude:
            exclude.remove(char)
            
# Add all words in the given keyword list to pre-defined token dictionary
multi_word = [w.split('_') for w in keywords_list ]   #if '_' in w 
tokenizer = MWETokenizer(multi_word)

,Effective action on climate change,Modern and resilient infrastructure,Pristine lakes and rivers,Clean drinking water,Safe and healthy communities,Low-carbon government,Clean energy,Sustainably managed lands and forests,Sustainable food,Clean growth,Healthy coasts and oceans,Healthy wildlife populations,Connecting Canadians with nature
0,climate-related_hazard,infrastructure,clean_lake,water,city,energy,energy,terrestrial_ecosystem,end_hunger,clean_enironment,ocean_resource,wildlife,nature
1,natural_disaster,resilient_infrastructure,healthy,clean,settlement,low-carbon,reliable,sustainably_manage_forest,nutritious,economy,ocean,wilderness,canadian
2,climate_change,facility,river,accessable,upgrade_slum,economy,sustainable,forest,sufficient,modern_world,sea,population,connecting
3,national_policy,economic_development,fraser,affordable,transport_system,clean_tech,renewable_energy,desertification,food,clean_technology,lake,plant,benefit
4,mitigation_action,industry,mackenzie,drinking,road_safety,cost_saving,global_energy_mix,reverse_landà_,malnutrition,greenhaouse_gas,coast,animal,community
5,green_climate,national_circumstance,great_lakes,hygiene,public_transport,carbon,energy_efficiency,land,stunting,healthier_community,water,environment,mental_health
6,climate_change_mitigation,small-scale_industrial,st_lawrence,water_pollution,urbanization,energy_efficiency,cleaner_fossil-fuel,degradation,wasting,reduce_ghg,coast,ecosystem,physical
7,change-related_planning,internet,churchill,dumping,human_settlement,efficient_product,investment_in_energy,biodiversity_loss,adolescent_girl,water_pollution,sea_level,food,improve
8,human_and_institutional_capacity_on_climate,wifi,variety,untreated_wastewater,natural_heritage,vehicle,clean_energy,inland_freshwaterà_,pregnant,clean_job,marine,medicine,child
9,critical_global_problem,information_access,plant,water-use,cultural_and_natural_heritage,equipment,land-locked_country,terrestrial,lactating_woman,growth,habibtat,flood,nature-based


In [7]:
def lemmatize_text(row): 
    text = str(row['caption_original'])
    #print(text)
    text = text.replace('’', '\'')
    tokens = tokenizer.tokenize(text.split())   
    # remove stop words
    stop_free = ' '.join(w for w in tokens if w not in stopWords and len(w) > 1)
    # remove punctuation
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    # lemmatize
    lemmas = ' '.join(lemma.lemmatize(word).lower() for word in punc_free.split() if len(lemma.lemmatize(word)) > 1)
    # remove stop words that appear after lemmatization
    stop_free_2 = ' '.join(w for w in lemmas.split() if w not in stopWords and len(w) > 1)
    #print(stop_free_2)
    return stop_free_2.split()

# assign a category based the max number of keywords found in each category
def find_category(row):
    text = row['lemmatized_text']
    keywords_found = []
    counter = {}
    for col in KEYWORDS_COLS:
        counter[col] = 0
    #print(counter)
    category = 'unknown'
    for word in text:
        for col in KEYWORDS_COLS:
            if word in category_dict[col]:
                #print(word)
                #print(category_dict[col])
                keywords_found.append(word)
                counter[col] += 1
    if len(keywords_found) > 0:
        category = max(counter.items(), key=operator.itemgetter(1))[0]    
    return keywords_found, category

In [8]:
pd.options.display.max_rows = 10
# read csv files and save targt columns to dataframe
filePaths = glob.glob(data_folder)  
for filename in filePaths:
    print(filename)
    basename = os.path.basename(filename)
    outputFileName = outputDir + basename
    data_df = pd.read_csv(filename, encoding = 'utf-8')
    if data_df.shape[0] < 1:
        data_df.to_csv(outputFileName, index=None) 
        continue
    #data_df['lang'] = data_df['caption_cleaned'].astype(str).apply(detect_lang)
    #data_df = data_df.drop(['category', 'words_matched_list'], axis=1)
    data_df.fillna('')
    #wrong_lang = data_df[data_df['lang'] != 'en'].shape[0]
    try:
        data_df['lemmatized_text'] = data_df.apply(lemmatize_text, axis=1)
    except:
        print('cannot process file: ' + basename)
        continue
    data_df['FSDS_matched_keywords'], data_df['FSDS_category'] = zip(*data_df.apply(find_category, axis=1))
    output_list = data_df.columns.tolist()
    output_list.remove('lemmatized_text')
    output_df = data_df[output_list]
    output_df.to_csv(outputFileName, index=None)    
    test_df = data_df[data_df['FSDS_category'] != 'unknown']
    display(test_df[['FSDS_matched_keywords', 'FSDS_category']])

./Accounts/output_chelsea/ParksCanada_tweets.csv


,FSDS_matched_keywords,FSDS_category
2,[park],Connecting Canadians with nature
5,[park],Connecting Canadians with nature
8,"[care, ecosystem, ecosystem, ecosystem, ecosys...",Healthy wildlife populations
10,"[forest, forest, park]",Clean drinking water
15,"[canadian, canadian, forest, forest]",Clean drinking water
...,...,...
752,"[canadian, canadian]",Sustainable food
755,[camping],Connecting Canadians with nature
762,"[development, development]",Modern and resilient infrastructure
764,"[safe, park]",Safe and healthy communities


./Accounts/output_chelsea/CanadianAgriculture_facebook_statuses.csv


,FSDS_matched_keywords,FSDS_category
0,"[canadian, canadian, greenhouse_gas, emission]",Low-carbon government
1,"[canadian, canadian]",Sustainable food
2,[crop],Healthy wildlife populations
3,"[canadian, canadian]",Sustainable food
6,"[canadian, canadian]",Sustainable food
...,...,...
1077,"[health, health, health]",Effective action on climate change
1079,[agri-food],Sustainable food
1080,"[canadian, canadian, food, food]",Sustainable food
1081,[agriculture],Pristine lakes and rivers


./Accounts/output_chelsea/CCG_GCC_tweets.csv


...

,FSDS_matched_keywords,FSDS_category
6,"[community, community]",Safe and healthy communities
8,[resilience],Safe and healthy communities
13,"[canadian, canadian, agriculture]",Pristine lakes and rivers
15,"[community, community]",Safe and healthy communities
19,"[agriculture, agri-food, industry]",Modern and resilient infrastructure
...,...,...
3094,[market],Sustainable food
3095,[benefit],Connecting Canadians with nature
3100,"[market, industry]",Modern and resilient infrastructure
3101,[research],Modern and resilient infrastructure


./Accounts/output_chelsea/ParksCanada_facebook_statuses.csv


,FSDS_matched_keywords,FSDS_category
0,[park],Connecting Canadians with nature
2,"[lake, lake, park]",Clean drinking water
3,[variety],Pristine lakes and rivers
4,"[marine, conservation, conservation]",Sustainably managed lands and forests
5,[park],Connecting Canadians with nature
...,...,...
1204,"[camping, camping, camping, food, food, camping]",Connecting Canadians with nature
1206,"[camping, park]",Connecting Canadians with nature
1215,"[camping, park, mountain, mountain, park]",Connecting Canadians with nature
1219,[park],Connecting Canadians with nature


./Accounts/output_chelsea/naturalresourcescanada_posts.csv


,FSDS_matched_keywords,FSDS_category
0,"[forest, forest, productivity, canadian, canad...",Sustainable food
1,"[land, infrastructure]",Modern and resilient infrastructure
6,"[ice, environment, environment]",Safe and healthy communities
7,"[healthy, healthy]",Pristine lakes and rivers
8,"[energy, energy, energy]",Low-carbon government
...,...,...
230,"[energy, energy, energy]",Low-carbon government
231,"[energy, energy, energy]",Low-carbon government
233,"[infrastructure, gas, vehicle]",Modern and resilient infrastructure
235,"[mountain, mountain]",Clean drinking water


In [10]:
print(exclude)

{'`', '}', '*', '=', '%', ':', '#', ']', '|', '@', ',', '$', ')', '.', '\\', '!', '[', "'", '^', '~', '{', '?', '(', '+', '&', '<', '>', ';', '"'}
